In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# =============================================================================
# Created By  : Huy Nguyen
# Created Date: Tue 7 Sep 2021
# =============================================================================

import requests
import pandas as pd
import plotly.express as px
import dash
import dash_table
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
from utils import *
from dash.dependencies import Input, Output, State

# Variables
API_ENDPOINTS = "https://api.covid19api.com/summary"
NAVBAR_LOGO = "https://cdn.pixabay.com/photo/2020/04/29/07/54/coronavirus-5107715_1280.png"
NAVBAR_TITLE = "Covid-19 Live Tracker"

######## GET DATA ###########
# Get data from API
response = requests.get(API_ENDPOINTS)
response.raise_for_status()
data = response.json()


######## GET COMPONENTS ###########
# Country
country_df = pd.DataFrame(data["Countries"])
fig = choropleth_plot(dataframe=country_df)

# Navigation bar
navbar = create_navbar(title=NAVBAR_TITLE, logo=NAVBAR_LOGO)

# Data Card
confirmed_card = make_card(data=data, case="Confirmed", title="Confirmed", color="secondary")
recovered_card = make_card(data=data, case="Recovered", title="Recovered", color="success")
deaths_card = make_card(data=data, case="Deaths", title="Deaths", color="danger")

cards = make_card_row(confirmed=confirmed_card, recovered=recovered_card, deaths=deaths_card)

# Get choropleth graph
choropleth_graph = dcc.Graph(id="choropleth", figure=fig)

# Table
table = create_table(data=data)

# Dropdown
dropdown = dcc.Dropdown(
    id= "dropdown",
    options=[{"label": con["Country"], "value": con["CountryCode"]} for con in data["Countries"]].append({"label": "All", "value": "all"}),
    value= "all",
    style= {"margin-bottom": "20px"}
)


# Graph + Table
graph_table = html.Div([dbc.Row([dbc.Col(choropleth_graph, ), dbc.Col(html.Div([dropdown, table]))])])


######## WRITE APP ###########
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    navbar,
    html.Div([cards, graph_table], style={"padding": "0px 200px"})
])


@app.callback(Output("table", "data"), [Input("dropdown", "value")])
def update_table(code):
    return get_rows(code, data)
        

######## RUN APP ###########
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Sep/2021 11:02:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2021 11:02:08] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2021 11:02:08] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2021 11:02:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2021 11:02:08] "GET /_dash-component-suites/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2021 11:02:08] "GET /_dash-component-suites/dash_table/async-table.js HTTP/1.1" 200 -


In [2]:
country_df


,ID,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,0caace6f-053d-4148-95b5-30de76173379,Afghanistan,AF,afghanistan,92,153626,3,7144,0,0,2021-09-08T01:34:24.512Z,{}
1,3c16cb81-cf3a-4e84-9922-a0f1309173cb,Albania,AL,albania,740,152239,4,2523,0,0,2021-09-08T01:34:24.512Z,{}
2,35586928-a0e7-4707-a19d-cfe32734aca8,Algeria,DZ,algeria,332,198645,26,5471,0,0,2021-09-08T01:34:24.512Z,{}
3,d6feca15-12d6-4bd2-a15e-15c6c1c01b8b,Andorra,AD,andorra,1,15070,0,130,0,0,2021-09-08T01:34:24.512Z,{}
4,2f87e1df-a615-4dbc-b280-1c5cce5168ff,Angola,AO,angola,0,48790,0,1282,0,0,2021-09-08T01:34:24.512Z,{}
...,...,...,...,...,...,...,...,...,...,...,...,...
187,61fa0657-2ff0-446d-8563-473dd2b2d8f6,Venezuela (Bolivarian Republic),VE,venezuela,1127,341314,17,4115,0,0,2021-09-08T01:34:24.512Z,{}
188,8d80e693-4c84-410f-95b5-e98f3f9ed1a3,Viet Nam,VN,vietnam,14208,550996,316,13701,0,0,2021-09-08T01:34:24.512Z,{}
189,1f119165-5390-4e84-bc6a-126fb807cf62,Yemen,YE,yemen,39,8181,4,1534,0,0,2021-09-08T01:34:24.512Z,{}
190,154e4df8-da3c-4790-844c-36cdbab3c710,Zambia,ZM,zambia,127,207294,3,3620,0,0,2021-09-08T01:34:24.512Z,{}


In [10]:
px.bar(country_df.sort_values(by= "NewConfirmed", ascending=False).head(20), x= "Country", y= "NewConfirmed", hover_data= ["CountryCode"], title= "Top 20 New Confirmed")